In [1]:
path_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"
path_results_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Results/"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from Tool_Functions.cleaning_data import *
from Tool_Functions.join_data import *
from Tool_Functions.test_comportment_reabo import *
from Tool_Functions.visual import *
from Tool_Functions.comportment_reabo import *

In [18]:
def ajout_anciennete(data_path, data_path_results):
    df = file_to_dataframe(data_path + 'df_Données_Reabos_odd_new_v2.csv')
    df['DATE_ACTE_REEL'] = pd.to_datetime(df['DATE_ACTE_REEL'])
    date_plus_ancienne = df.groupby('ID_ABONNE')['DATE_ACTE_REEL'].min()
    date_reference = pd.to_datetime('2023-10-30')
    df['PREMIERE_APPARITION'] = df['ID_ABONNE'].map(date_plus_ancienne)
    df['ANCIENNETE'] = (date_reference - df['PREMIERE_APPARITION']).dt.days
    save_to_csv_file(df, data_path + 'df_Données_Reabos_odd_final.csv')
    return True

In [3]:
df1 = file_to_dataframe(path_antoine + 'df_Donnees_Reabos_odd_new_v2.csv')

/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Statapp-CANAL-1/Data_operations/Tool_Functions/cleaning_data.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  datas = pd.read_csv(filenames,delimiter=st)


In [6]:
df1['DATE_ACTE_REEL'] = pd.to_datetime(df1['DATE_ACTE_REEL'])

In [12]:
date_plus_ancienne = df1.groupby('ID_ABONNE')['DATE_ACTE_REEL'].min()
date_reference = pd.to_datetime('2023-10-30')

In [10]:
df1['PREMIERE_APPARITION'] = df1['ID_ABONNE'].map(date_plus_ancienne)

In [15]:
df1['ANCIENNETE'] = (date_reference - df1['PREMIERE_APPARITION']).dt.days

In [19]:
save_to_csv_file(df1, path_antoine + 'df_Données_Reabos_odd_final.csv')

True

In [7]:
df = file_to_dataframe(path_antoine + 'fusion_table_score_v1.csv')

In [ ]:
df.head()

In [ ]:
scaler = StandardScaler()

In [5]:
new_df = file_to_dataframe(path_antoine + 'new_datas_diff_%.csv')
df = new_df.drop(columns = ['NB_APPARITIONS', 'MOY_DELAI'])

In [ ]:
df_original = pd.DataFrame(df).set_index('ID_ABONNE')
columns_names = df_original.columns
columns_names

In [7]:
# Supposons que df est votre DataFrame et qu'il contient des valeurs 'inf'
# Remplacer 'inf' par NaN pour le traitement ultérieur
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
# Séparer la colonne ID_ABONNE
df_id = df[['ID_ABONNE']].copy()

# Séparer les données à normaliser (sans la colonne ID_ABONNE)
df_to_scale = df.drop(columns=['ID_ABONNE'])


In [9]:

# Mise à l'échelle des données sans ID_ABONNE
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_to_scale.fillna(0)), columns=df_to_scale.columns)

# Concaténation de la colonne ID_ABONNE avec les données normalisées
df_scaled = pd.concat([df_id, df_scaled], axis=1)
df_scaled[np.isnan(df)] = -3

In [ ]:
df_scaled

In [12]:
# Initialiser le modèle KMeans avec un nombre déterminé de clusters
kmeans = KMeans(n_clusters=7) 
kmeans.fit(df_scaled)
clusters = kmeans.labels_

Hyperparamètres : 
init : Méthode d'initialisation des centroïdes. Les options courantes sont 'k-means++' (défaut), 'random', un tableau donné de formes (n_clusters, n_features) ou une méthode d'initialisation personnalisée. 'k-means++' choisit les centroïdes initiaux pour le clustering k-means d'une manière qui accélère la convergence.

n_init : Nombre de fois où l'algorithme k-means sera exécuté avec différentes graines de centroïde. Les résultats finaux seront la meilleure sortie de n_init exécutions consécutives en termes d'inertie.

max_iter : Le nombre maximal d'itérations de l'algorithme k-means pour une seule exécution.
tol : Tolérance pour déclarer la convergence. Si les changements dans les centroïdes sont inférieurs à cette tolérance, l'algorithme peut arrêter plus tôt.

precompute_distances : Détermine si les distances entre les points doivent être précalculées ou calculées à la volée. Cela peut accélérer les choses mais peut aussi consommer beaucoup de mémoire pour les grands ensembles de données.
verbose : Mode verbeux.

random_state : Graine utilisée par le générateur de nombres aléatoires pour l'initialisation du centroïde ou si init est une graine pour l'algorithme k-means++.

copy_x : Si True (par défaut), les données d'entrée sont copiées, sinon elles peuvent être écrasées pendant le clustering.

algorithm : Choix de l'algorithme pour calculer les k-moyens parmi 'auto', 'full' ou 'elkan'. Le choix classique 'full' est l'algorithme EM pour le k-means. 'elkan' est une variante plus efficace qui est plus rapide sur les ensembles de données avec de nombreux clusters mais qui ne fonctionne pas avec des métriques autres que la distance euclidienne.

In [17]:
df['Cluster_KMeans'] = kmeans.labels_

In [18]:
compte = df.groupby('Cluster_KMeans').count()
compte

In [ ]:
centroids = kmeans.cluster_centers_
centroids
# Convertir les centroïdes en DataFrame pour une meilleure lisibilité
centroids_df = pd.DataFrame(centroids, columns=columns_names)
# Afficher les centroïdes
print(centroids_df)
save_to_csv_file(centroids_df, path_antoine + 'centres_clusters.csv')


In [ ]:
df_scaled_df = pd.DataFrame(df_scaled, columns=columns_names)
df_scaled_df['Cluster_KMeans'] = kmeans.labels_
k = 7
for i in range(k):
    print(f"Cluster {i} characteristics:")
    cluster_data = df_scaled_df[df_scaled_df['Cluster_KMeans'] == i]
    # Vous pouvez ici calculer des statistiques ou des graphiques pour comprendre chaque cluster
    print(cluster_data.describe())  # Résumé statistique
    save_to_csv_file(cluster_data.describe(), path_antoine + 'cluster_data.csv')